In [1]:
import pandas as pd
import numpy as np
import pymc as pm
from src.data_utils import load_and_process_data
from src.model import run_full_analysis
from src.trace_save_load import load_previous_season_trace, extract_previous_season_priors
import matplotlib.pyplot as plt

league = ['Premier_League', 'Championship']
match_date = '2024-08-01'


# Simple one-liner to get everything ready
df, team_mapping, n_teams = load_and_process_data(
    db_path=r'C:\Users\Owner\dev\algobetting\infra\data\db\fotmob.db',
    league=league, 
    #season=season,
    match_date=match_date,
    max_goals=9,
    decay_rate=0.002,
    goals_weight=0.5,
    xg_weight=0.5,
    psxg_weight=0.3

)

df

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


,match_id,league_id,match_date,home_team,away_team,home_goals,away_goals,weight,days_ago,is_actual,poisson_binomial_xg_prob,poisson_binomial_psxg_prob,poisson_xg_total_prob,home_idx,away_idx
0,4506263,Premier_League,2024-08-16,Man United,Fulham,0,0,0.007787,393,False,0.020302,0.007456,0.057049,22,14
1,4506263,Premier_League,2024-08-16,Man United,Fulham,0,1,0.004343,393,False,0.009283,0.015801,0.024834,22,14
2,4506263,Premier_League,2024-08-16,Man United,Fulham,0,2,0.001072,393,False,0.001888,0.005104,0.005405,22,14
3,4506263,Premier_League,2024-08-16,Man United,Fulham,1,0,0.275488,393,True,0.112064,0.062784,0.138839,22,14
4,4506263,Premier_League,2024-08-16,Man United,Fulham,1,1,0.017755,393,False,0.051241,0.133064,0.060439,22,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26973,4813414,Premier_League,2025-09-13,West Ham,Tottenham,3,1,0.000963,0,False,0.004998,0.000144,0.008579,42,39
26974,4813414,Premier_League,2025-09-13,West Ham,Tottenham,3,2,0.000634,0,False,0.003315,0.000297,0.005527,42,39
26975,4813414,Premier_League,2025-09-13,West Ham,Tottenham,3,3,0.000278,0,False,0.001285,0.000220,0.002374,42,39
26976,4813414,Premier_League,2025-09-13,West Ham,Tottenham,4,0,0.000120,0,False,0.000176,0.000000,0.001084,42,39


In [2]:
model, trace, results = run_full_analysis(
    train_df=df,
    teams=list(team_mapping.keys()),  
    season=2025,
    n_teams=n_teams,
    trace_samples=5000,
    tune_samples=3000,
)

Season: 2025
Teams: 44
Model version: v1
Samples: 5000 (tune: 3000)

Building and sampling model...


TypeError: build_and_sample_model() missing 1 required positional argument: 'prem_team_indices'